In [40]:
import numpy as np
import pandas as pd
import math
from imblearn.over_sampling import RandomOverSampler
import pymorphy2, re

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from keras.layers import Input, Embedding, Activation, Flatten, Dense, concatenate
from keras.layers import Conv1D, MaxPooling1D, Dropout, LSTM
from keras.models import Model

from nltk.corpus import stopwords

Using TensorFlow backend.


In [86]:
path1='pairs_rus.csv'
path2='not_pairs_rus.csv'

In [87]:
df1=pd.read_csv(path1, sep=';', encoding='cp1251')
df2=pd.read_csv(path2, sep=';', encoding='cp1251')

In [88]:
z=np.zeros(df1.shape[0])
z.shape=(df1.shape[0],1)
df1=pd.DataFrame(np.hstack((df1.values,z)))

In [89]:
z=np.ones(df2.shape[0])
z.shape=(df2.shape[0],1)
df2=pd.DataFrame(np.hstack((df2.values,z)))            

In [90]:
df1=pd.DataFrame(np.vstack((df1.values, df2.values)))
df1=df1.sample(frac=1)
df1

,0,1,2
104133,Собаки играют в воде,А мальчик серфинга на волне,1
131147,Несколько хоккеиста из разных команд и судьи б...,Мальчик сделать скейтборд трюк на перилах,1
115073,Люди в закрытый карнавал особенность игры с бо...,Четыре человека в различные стадии санках по с...,1
162108,"Коричневая собака Чейз что-то человек, за ним ...",Загар и белая собака бежит в воду,1
39304,Человека сфотографировать в толпе,Два человека в толпе будет развернуться сфотог...,0
55077,два человека будут таскать желтый плот в реку,Два человека в пруд спустить спасательный плот,0
3330,мальчик в полосатой рубашки будет играть с фут...,Мальчик прыжок в воздух в сторону футбольный мяч,0
104642,Человек серфинг большая волна,Четыре женщины носят вечерние платья создают в...,1
47845,Четыре дамы в купальниках играют в волейбол на...,"Девушки в бикини играют в волейбол, рядом водоем",0
65639,Собака будет прыгать в воздухе над какой-то ли...,Маленькая коричневая собака быть в окружении г...,0


In [91]:
m = df1.values.shape[0]
train_length = int(0.7 * m)
train_data1, dev_data1 = df1[0].values[:train_length],\
    df1[0].values[train_length:]
train_data2, dev_data2 = df1[1].values[:train_length],\
    df1[1].values[train_length:]
train_classes, dev_classes = df1[2].values[:train_length],\
    df1[2].values[train_length:]

In [41]:
from gensim.models.keyedvectors import KeyedVectors
from tqdm import tqdm

print("считываю кутузовский файл")
path = r"C:\Users\Annie\Downloads\ruwikiruscorpora_upos_skipgram_300_2_2018.vec\ruwikiruscorpora_upos_skipgram_300_2_2018.vec"
w2v = KeyedVectors.load_word2vec_format(path, binary=False)
print("считал")

C:\Users\Annie\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


считываю кутузовский файл


C:\Users\Annie\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


считал


In [82]:
import os
import pymorphy2
import re
import pandas as pd
import keras
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

ma = pymorphy2.MorphAnalyzer()

def check_in_kutuzov(word):
    part_of_speech = str(ma.parse(word)[0].tag.POS).upper()
    if part_of_speech == 'ADJF':
        part_of_speech = "ADJ"
    elif part_of_speech == 'ADJS':
        part_of_speech = "ADJ"
    elif part_of_speech == 'NUMR':
        part_of_speech = "NUMR"
    elif part_of_speech == 'ADVB':
        part_of_speech == 'ADV'
    elif part_of_speech == 'INFN' or part_of_speech == 'PRTF' or part_of_speech == 'PRTS':
        part_of_speech = 'VERB'

    new_word = word + "_" + part_of_speech
    if new_word in w2v.vocab:
        return new_word
    else:
        return ""

def clean_text(text):
    text = text.replace("\\", " ").replace(u"╚", " ").replace(u"╩", " ")
    text = text.lower()
    text = re.sub('\-\s\r\n\s{1,}|\-\s\r\n|\r\n', '', text) #deleting newlines and line-breaks
    text = re.sub('[.,:;_%©?*,!@#$%^&()\d]|[+=]|[[]|[]]|[/]|"|\s{2,}|-', ' ', text) #deleting symbols 
    text = " ".join(ma.parse(word)[0].normal_form for word in text.split())
    text = ' '.join(word for word in text.split() if len(word)>3)
    all_words = text.split(' ')
    all_correct_words = ""
    for word in all_words:
        new_word = check_in_kutuzov(word)
        if new_word !="":
            if all_correct_words=="":
                all_correct_words += new_word
            else:
                all_correct_words += " "+new_word
    #text = text.encode("utf-8")
    return all_correct_words

In [93]:
print('train')
for i in range(len(train_data1)):
    train_data1[i]=clean_text(train_data1[i])
print('dev')
for i in range(len(dev_data1)):
    dev_data1[i]=clean_text(dev_data1[i])
print('train')
for i in range(len(train_data2)):
    train_data2[i]=clean_text(train_data2[i])
print('dev')
for i in range(len(dev_data2)):
    dev_data2[i]=clean_text(dev_data2[i])

train
dev
train
dev


In [94]:
train_classes=to_categorical(train_classes)
dev_classes=to_categorical(dev_classes)

In [95]:
num_classes = 2

# Максимальное количество слов в самом длинном тексте
max_words = 0

for desc in train_data1:
    words = len(desc.split())
    if words > max_words:
        max_words = words
for desc in train_data2:
    words = len(desc.split())
    if words > max_words:
        max_words = words
print('Максимальное количество слов в самом длинном тексте: {} слов'.format(max_words))

maxSequenceLength = max_words

Максимальное количество слов в самом длинном тексте: 18 слов


In [108]:
for i in range(len(train_data1)):
    train_data1[i]=train_data1[i].replace('NOUN','NOUN ').replace('VERB','VERB ').replace('ADJ','ADJ ').replace('  ',' ')

In [109]:
for i in range(len(train_data2)):
    train_data2[i]=train_data2[i].replace('NOUN','NOUN ').replace('VERB','VERB ').replace('ADJ','ADJ ').replace('  ',' ')
for i in range(len(dev_data1)):
    dev_data1[i]=dev_data1[i].replace('NOUN','NOUN ').replace('VERB','VERB ').replace('ADJ','ADJ ').replace('  ',' ')
for i in range(len(dev_data2)):
    dev_data2[i]=dev_data2[i].replace('NOUN','NOUN ').replace('VERB','VERB ').replace('ADJ','ADJ ').replace('  ',' ')

In [110]:
train_data1[0:1000]

array(['собакаNOUN игратьVERB водаNOUN ',
       'хоккеистNOUN разныйADJ командаNOUN судьяNOUN ',
       'человекNOUN закрытыйADJ карнавалNOUN особенностьNOUN играNOUN большойADJ чучелоNOUN животноеADJ качествоNOUN призNOUN ',
       'коричневыйADJ собакаNOUN человекNOUN броситьсяVERB пляжNOUN ',
       'человекNOUN сфотографироватьVERB толпаNOUN ',
       'человекNOUN таскатьVERB жёлтыйADJ плотNOUN рекаNOUN ',
       'мальчикNOUN полосатыйADJ рубашкаNOUN игратьVERB футбольныйADJ травянистыйADJ паркаNOUN кустNOUN ',
       'человекNOUN серфингNOUN большойADJ волнаNOUN ',
       'дамаNOUN купальникNOUN игратьVERB волейболNOUN пляжNOUN ',
       'собакаNOUN прыгатьVERB воздухNOUN листокNOUN ',
       'человекNOUN оранжевыйADJ футболкаNOUN бегатьVERB улицаNOUN ',
       'женскийADJ искусствоNOUN демонстрироватьVERB ударNOUN аудиторияNOUN ',
       'велосипедистNOUN попыткаNOUN трюкNOUN полётNOUN открытыйADJ воздухNOUN лестницаNOUN ',
       'собакаNOUN держатьVERB ротаNOUN ',
       'соба

In [98]:
import pickle
with open('train_data1.dat', "wb") as file:
        pickle.dump(train_data1, file)
with open('train_data2.dat', "wb") as file:
        pickle.dump(train_data2, file)
with open('dev_data1.dat', "wb") as file:
        pickle.dump(dev_data1, file)
with open('dev_data2.dat', "wb") as file:
        pickle.dump(dev_data2, file)

In [56]:
with open('train_data1.dat', "rb") as file:
        train_data1=pickle.load(file)
with open('train_data2.dat', "rb") as file:
        train_data2=pickle.load(file)
with open('dev_data1.dat', "rb") as file:
        dev_data1=pickle.load(file)
with open('dev_data2.dat', "rb") as file:
        dev_data2=pickle.load(file)

In [101]:
for i in range(len(train_data1)):
    train_data1[i] = train_data1[i].replace('_','')
for i in range(len(train_data2)):
    train_data2[i] = train_data2[i].replace('_','')
for i in range(len(dev_data1)):
    dev_data1[i] = dev_data1[i].replace('_','')
for i in range(len(dev_data2)):
    dev_data2[i] = dev_data2[i].replace('_','')

In [111]:
t = Tokenizer()
t.fit_on_texts(train_data1+train_data2)
vocab_size = len(t.word_index) + 1

encoded_docs_train1 = t.texts_to_sequences(train_data1)
encoded_docs_dev1 = t.texts_to_sequences(dev_data1)
encoded_docs_train2 = t.texts_to_sequences(train_data2)
encoded_docs_dev2 = t.texts_to_sequences(dev_data2)

padded_docs_train1 = sequence.pad_sequences(encoded_docs_train1, maxlen=maxSequenceLength)
padded_docs_dev1 = sequence.pad_sequences(encoded_docs_dev1, maxlen=maxSequenceLength)
padded_docs_train2 = sequence.pad_sequences(encoded_docs_train2, maxlen=maxSequenceLength)
padded_docs_dev2 = sequence.pad_sequences(encoded_docs_dev2, maxlen=maxSequenceLength)

In [112]:
t.word_index

{'собакаnoun': 1,
 'человекnoun': 2,
 'чёрныйnoun': 3,
 'коричневыйadj': 4,
 'белыйadj': 5,
 'игратьverb': 6,
 'женщинаnoun': 7,
 'прыгатьverb': 8,
 'водаnoun': 9,
 'траваnoun': 10,
 'бежатьverb': 11,
 'мужчинаnoun': 12,
 'красныйadj': 13,
 'носитьverb': 14,
 'воздухnoun': 15,
 'большойadj': 16,
 'бегатьverb': 17,
 'снегnoun': 18,
 'стоятьverb': 19,
 'велосипедnoun': 20,
 'смотретьverb': 21,
 'другnoun': 22,
 'мальчикnoun': 23,
 'зелёныйadj': 24,
 'группаnoun': 25,
 'жёлтыйadj': 26,
 'сидетьverb': 27,
 'девушкаnoun': 28,
 'ребёнокnoun': 29,
 'времяnoun': 30,
 'рубашкаnoun': 31,
 'прыжокnoun': 32,
 'маленькийadj': 33,
 'трюкnoun': 34,
 'пляжnoun': 35,
 'держатьverb': 36,
 'молодаяnoun': 37,
 'пойматьverb': 38,
 'ездитьverb': 39,
 'синийadj': 40,
 'ротаnoun': 41,
 'игрушкаnoun': 42,
 'палкаnoun': 43,
 'сделатьverb': 44,
 'оранжевыйadj': 45,
 'улицаnoun': 46,
 'ходитьverb': 47,
 'гонкаnoun': 48,
 'рукаnoun': 49,
 'волнаnoun': 50,
 'грязьnoun': 51,
 'теннисныйadj': 52,
 'мотоциклnoun': 53,

In [113]:
total_unique_words = len(t.word_counts)
print('Всего уникальных слов в словаре: {}'.format(total_unique_words))

Всего уникальных слов в словаре: 5019


In [114]:
print("Создаю матрицу")
embedding_matrix = np.zeros((vocab_size, 300))
print("Начинаем")
for word, i in t.word_index.items():
    
    try:
        new_word = word.replace('adj', '_ADJ')
        new_word = new_word.replace('noun', '_NOUN')
        new_word = new_word.replace('verb', '_VERB')
        embedding_vector = w2v[new_word]

        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    except:
        print(word)
        print(i) 

Создаю матрицу
Начинаем
стопintj
2260


In [115]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, concatenate
from keras.optimizers import RMSprop
from keras.layers import Bidirectional

inputs_x=Input(shape=(maxSequenceLength,), name='input_1', dtype='int64')
inputs_y=Input(shape=(maxSequenceLength,), name='input_2', dtype='int64')

x=Embedding(vocab_size, 300, name='word2vec_1', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_x)
y=Embedding(vocab_size, 300, name='word2vec_2', weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)(inputs_y)

x=Conv1D(128,7, padding='same', name='conv_1')(x)
x=MaxPooling1D(pool_size = (3), name='pooling_1')(x)
y=Conv1D(128,7, padding='same', name='conv_2')(y)
y=MaxPooling1D(pool_size = (3), name='pooling_2')(y)

x=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='bltsm_1')(x)
y=LSTM(64, dropout=0.5, recurrent_dropout=0.2, name='bltsm_2')(y)

x=Model(inputs=inputs_x, outputs=x)
y=Model(inputs=inputs_y, outputs=y)

combined=concatenate([x.output, y.output], name='concat')

ex=Dense(32, activation='tanh', name='fully_connected')(combined)
predictions=Dense(2, activation='softmax', name='output')(ex)
model=Model(inputs=[inputs_x, inputs_y], outputs=predictions)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 18)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 18)           0                                            
__________________________________________________________________________________________________
word2vec_1 (Embedding)          (None, 18, 300)      1506000     input_1[0][0]                    
_________________________________________________________________________________________________

In [ ]:
history = model.fit([padded_docs_train1, padded_docs_train2], train_classes, epochs = 10, verbose=2, validation_data=([padded_docs_dev1, padded_docs_dev2], dev_classes))

Train on 113933 samples, validate on 48829 samples
Epoch 1/10
 - 531s - loss: 0.6894 - acc: 0.5305 - val_loss: 0.6808 - val_acc: 0.5787
Epoch 2/10
 - 616s - loss: 0.6403 - acc: 0.6054 - val_loss: 0.4669 - val_acc: 0.7710
Epoch 3/10


In [ ]:
df_test=pd.read_csv('test_pairs.csv', sep=';', encoding='cp1251')
df_test

In [ ]:
test_data1=df[0].values
test_data2=df[1].values

for i in range(len(test_data1)):
    test_data1[i]=clean_text(test_data1[i])
for i in range(len(test_data2)):
    test_data2[i]=clean_text(test_data2[i])
    
for i in range(len(test_data1)):
    test_data1[i] = test_data1[i].replace('_','')
for i in range(len(test_data2)):
    test_data2[i] = test_data2[i].replace('_','')
    
encoded_docs_test1 = t.texts_to_sequences(test_data1)
encoded_docs_test2 = t.texts_to_sequences(test_data2)

padded_docs_test1 = sequence.pad_sequences(encoded_docs_test1, maxlen=maxSequenceLength)
padded_docs_test2 = sequence.pad_sequences(encoded_docs_test2, maxlen=maxSequenceLength)

predict = np.argmax(model.predict([padded_docs_test1, padded_docs_test2]), axis=1)
print(predict)